In [23]:
import pandas as pd
import tensorflow as tf
import re
import nltk
import preprocessor as p
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [24]:
df = pd.read_csv('idk.csv')

In [25]:
df['Text'][0]

'rwanda is set to host the headquarters of united nations development programmes undp new innovation financing facilityfind out more rwanda'

Tokenize the sentence

In [26]:
vocab_size = 20000
embedding_dim = 32
max_length = 300
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

In [27]:
training_size = 20000
training_sentences = df['Text'][0:training_size].to_list()
testing_sentences = df['Text'][training_size:].to_list()
training_labels = df['Label'][0:training_size].to_list()
testing_labels = df['Label'][training_size:].to_list()

In [28]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [53]:
tokenizer

In [29]:
# Need this block to get it to work with TensorFlow 2.x
import numpy as np
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

In [30]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [31]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 300, 32)           640000    
                                                                 
 flatten_2 (Flatten)         (None, 9600)              0         
                                                                 
 dense_4 (Dense)             (None, 128)               1228928   
                                                                 
 dense_5 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,869,057
Trainable params: 1,869,057
Non-trainable params: 0
_________________________________________________________________


In [32]:
num_epochs = 4
history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=1)

Epoch 1/4
625/625 [==============================] - 17s 26ms/step - loss: 0.2669 - accuracy: 0.8705 - val_loss: 0.0966 - val_accuracy: 0.9624
Epoch 2/4
625/625 [==============================] - 16s 26ms/step - loss: 0.0509 - accuracy: 0.9823 - val_loss: 0.1025 - val_accuracy: 0.9598
Epoch 3/4
625/625 [==============================] - 17s 27ms/step - loss: 0.0227 - accuracy: 0.9933 - val_loss: 0.1252 - val_accuracy: 0.9549
Epoch 4/4
625/625 [==============================] - 16s 25ms/step - loss: 0.0123 - accuracy: 0.9969 - val_loss: 0.1332 - val_accuracy: 0.9551


In [33]:
import time

t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(current_time)

model.save(f"./saved_models/simple_ann({current_time}).h5")

16:03:47


In [34]:
from keras import  Sequential
from keras.layers import Embedding, LSTM, Bidirectional, Conv1D, GlobalMaxPooling1D, Dense
cnn_model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(1, activation='sigmoid')
])


cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(cnn_model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 300, 32)           640000    
                                                                 
 conv1d (Conv1D)             (None, 296, 128)          20608     
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense_6 (Dense)             (None, 1)                 129       
                                                                 
Total params: 660,737
Trainable params: 660,737
Non-trainable params: 0
_________________________________________________________________
None


In [35]:
num_epochs = 10
history_cnn = cnn_model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=1)

Epoch 1/10
625/625 [==============================] - 14s 21ms/step - loss: 0.1698 - acc: 0.9226 - val_loss: 0.0723 - val_acc: 0.9691
Epoch 2/10
625/625 [==============================] - 13s 22ms/step - loss: 0.0430 - acc: 0.9861 - val_loss: 0.0778 - val_acc: 0.9680
Epoch 3/10
625/625 [==============================] - 13s 20ms/step - loss: 0.0229 - acc: 0.9932 - val_loss: 0.0895 - val_acc: 0.9691
Epoch 4/10
625/625 [==============================] - 13s 20ms/step - loss: 0.0120 - acc: 0.9966 - val_loss: 0.1036 - val_acc: 0.9648
Epoch 5/10
625/625 [==============================] - 12s 20ms/step - loss: 0.0072 - acc: 0.9982 - val_loss: 0.1114 - val_acc: 0.9657
Epoch 6/10
625/625 [==============================] - 12s 19ms/step - loss: 0.0051 - acc: 0.9985 - val_loss: 0.1192 - val_acc: 0.9645
Epoch 7/10
625/625 [==============================] - 12s 19ms/step - loss: 0.0045 - acc: 0.9987 - val_loss: 0.1273 - val_acc: 0.9638
Epoch 8/10
625/625 [==============================] - 12s 19ms

# LSTM model

In [36]:

lstm_model = tf.keras.Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    Bidirectional(LSTM(128)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [37]:
lstm_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 300, 32)           640000    
                                                                 
 bidirectional (Bidirectiona  (None, 256)              164864    
 l)                                                              
                                                                 
 dense_7 (Dense)             (None, 1)                 257       
                                                                 
Total params: 805,121
Trainable params: 805,121
Non-trainable params: 0
_________________________________________________________________


In [38]:
# num_epochs = 10
# lstm_model_histroy = lstm_model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=1)

In [41]:
def prediction(s):
    sequences = tokenizer.texts_to_sequences([s])
    padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
    # x = 0
    # if model_name == 'lstm':
    y = cnn_model.predict(padded)
    # elif model_name == 'ann':
    x = model.predict(padded)
      
    return f"ANN: {x}, CNN: {y}"
    # if x > 0.7:
    #     return f"Positive: confidence {x}" 
    # elif x < 0.33:
    #     return f"Negative: confidence {x}"
    # return f"Can't say: confidence {x}"

In [50]:
prediction("i am having a mental breakdown")

1/1 [==============================] - 0s 41ms/step


'ANN: [[0.10401616]], CNN: [[0.05187383]]'

In [54]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, './folder')

ModuleNotFoundError: No module named 'tensorflowjs'